In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import time

In [2]:
API = "RGAPI-56ab2022-2df5-4866-8ebc-30b500d85fa1"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API
}


# 챌린저 유저 기본정보 가져오기

In [24]:
challenger_URL = "https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"
res = req.get(challenger_URL, headers=headers)

# 전처리 후 데이터프레임 생성
challenger_df = pd.DataFrame(res.json())
challenger_entries_df = pd.DataFrame(dict(challenger_df.entries)).T

challenger_df = pd.concat([challenger_df, challenger_entries_df], axis=1)
challenger_df.drop(["queue", "name", "entries"], axis=1, inplace=True)

# 데이터프레임 출력
print(challenger_df.shape)
challenger_df.head()

(300, 12)


,tier,leagueId,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,CHALLENGER,5cc4fe58-4e57-37f1-9dc3-7c32b2632923,3ATf4fExEgsujITIsfZ39_uj9HO6ZnJbmdmG40qVkivZI2Y,테디봇1호,693,I,442,401,False,False,False,False
1,CHALLENGER,5cc4fe58-4e57-37f1-9dc3-7c32b2632923,yu4bXchLYXbjEv_Ky_-MDfnXzvLEBQKJ0ewgM2Jd8LeuRmQ,DRX Peach,697,I,477,407,False,False,True,False
2,CHALLENGER,5cc4fe58-4e57-37f1-9dc3-7c32b2632923,DXJu8toUsNRHzYIbmPC1tuE_S2jXVgDVxLrR1BMeWHqNetY,gkstmdals,948,I,641,530,True,False,False,True
3,CHALLENGER,5cc4fe58-4e57-37f1-9dc3-7c32b2632923,tYbkQMevsN43mWV0IbXpFNeaS6o1Dzs8jCkxIBaDWoEdnxE,HLE Vsta,1045,I,262,211,True,False,False,False
4,CHALLENGER,5cc4fe58-4e57-37f1-9dc3-7c32b2632923,nEweCtYc43gustbTPjuC39GymzE4KTt_xTkx5SAZHCCD-2A,Awe Smash,837,I,291,240,False,False,False,False


# 챌린저 유저 account 가져오기

In [25]:
c_id = challenger_df.summonerId

In [27]:
c_id[0]

'3ATf4fExEgsujITIsfZ39_uj9HO6ZnJbmdmG40qVkivZI2Y'

In [29]:
# 유저들의 account를 저장할 리스트 생성
user_account = []

tally = 0  # 제대로 작동하고 있는지 중간중간 출력해주기 위함
for ele in c_id:
    try:
        URL = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/{}".format(ele)
        res = req.get(URL, headers=headers)
        
        # API 이용 제한 (2분에 100개) 고려해주기
        while res.status_code == 429:
            print("API 이용량 제한. 30초 대기")
            time.sleep(30)
            res = req.get(URL, headers=headers)
        
        # 챌린저 ID 리스트에 추가
        user_account.append(res.json()["accountId"])
        tally += 1
    
    except:
        print("Unknown Error")
    
    if tally%50 == 0:
        print("{}개의 user account 가져오기 성공".format(tally))

50개의 user account 가져오기 성공
100개의 user account 가져오기 성공
API 이용량 제한. 30초 대기
API 이용량 제한. 30초 대기
API 이용량 제한. 30초 대기
API 이용량 제한. 30초 대기
150개의 user account 가져오기 성공
200개의 user account 가져오기 성공
API 이용량 제한. 30초 대기
API 이용량 제한. 30초 대기
API 이용량 제한. 30초 대기
API 이용량 제한. 30초 대기
250개의 user account 가져오기 성공
300개의 user account 가져오기 성공


In [32]:
pd.DataFrame(data={"account": user_account}).to_csv("account.csv")